In [11]:
# Enable auto-reloading of external modules - useful during development
%load_ext autoreload
%autoreload 2

# Configure Python path to find our custom modules
import sys
from pathlib import Path

# Add project root to the Python path for proper imports
project_root = Path.cwd().parent
if project_root not in sys.path:
    sys.path.insert(0, str(project_root))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Import necessary libraries
import src.processing as processing
import src.config as lists

In [13]:
# Load and process data
df = processing.load_data("/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv")
df_prepared = processing.prepare_data(df)
df_added_features = processing.create_all_model_features_orchestrated(df_prepared)
df_missing = processing.drop_missing_final_vars_streamlined(df_added_features, lists.final_set_A_predictor_names_and_dependent)
df_final = processing.annual_winsorize_variables(df_missing, lists.columns_to_winsorize)

/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/src/processing.py:41: DtypeWarning: Columns (10,11,12,16,17,22,26,30,33,34,35,945,946,948,949,950,955,969,970,975,980) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Data loaded successfully from /Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv
Original number of observations: 317304
Number of columns after selection: 30
Observations after year filter (2000-2023): 302751
Observations after excluding financial and utility firms: 170598
Starting feature construction. Initial df shape: (170598, 30)
  Creating lags for: ['at', 'ni', 'rect', 'invt', 'ap', 'sale']

Performing pre-calculation validity checks & preparations...
  Missing 'xrd' values filled with 0.
  'ipo_year' created from 'ipodate'.

Constructing dependent variable...
  OCF_Scaled_t_plus_1 created.

Constructing Set A (OLS) predictors...
  Set A predictors constructed.

Constructing control dummy variables...
  Dummy variables constructed.

Constructing Set B (additional ML) predictors...
  Set B predictors constructed.

Selecting final model variables and dropping intermediate columns...
  Shape of DataFrame 

In [14]:
# Split data chronologically - train on pre-2018, test on 2018+
train_df, test_df = processing.split_data_chronologically(df_final, 'fyear', split_year=2018)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training period: {train_df['fyear'].min()} - {train_df['fyear'].max()}")
print(f"Test period: {test_df['fyear'].min()} - {test_df['fyear'].max()}")

Training set: 102567 obs (Predictor years <= 2018)
Test set: 20882 obs (Predictor years > 2018)
Training data shape: (102567, 31)
Test data shape: (20882, 31)
Training period: 2001.0 - 2018.0
Test period: 2019.0 - 2022.0


In [ ]:
# Prepare features and dependent variable for Random Forest regression
X_train = train_df[lists.SET_A_FEATURES]
y_train = train_df[lists.DEPENDENT_VARIABLE]

X_test = test_df[lists.SET_A_FEATURES]
y_test = test_df[lists.DEPENDENT_VARIABLE]

print("Features included in the model:")
for i, feature in enumerate(X_train.columns, 1):
    print(f"{i:2d}. {feature}")

print(f"\nDependent variable: {lists.DEPENDENT_VARIABLE}")
print(f"Training observations: {len(X_train)}")
print(f"Test observations: {len(X_test)}")

# Hyperparameter optimization using RandomizedSearchCV
print("Performing hyperparameter optimization for Random Forest...")
print("Using out-of-bag samples for validation...")

start_time = time.time()

# Use RandomizedSearchCV for efficient hyperparameter search
# Limited to 20 iterations for computational efficiency (total parameter combinations: 48)
random_search = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=param_distributions,
    n_iter=20,  # Limited iterations for computational efficiency
    cv=3,       # 3-fold cross-validation for performance evaluation
    scoring='r2',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train, y_train)

# Extract optimal parameters and model
best_params = random_search.best_params_
best_score = random_search.best_score_
best_model = random_search.best_estimator_

Features included in the model:
 1. OCF_Scaled_Lag_t
 2. NI_Scaled_t
 3. Accruals_Scaled_t
 4. Delta_Rec_Scaled_t
 5. Delta_Inv_Scaled_t
 6. Delta_AP_Scaled_t
 7. DP_Scaled_t
 8. ln_at_t

Dependent variable: OCF_Scaled_t_plus_1
Training observations: 102567
Test observations: 20882


In [16]:
# =============================================================================
# OOB VALIDATION FOR HYPERPARAMETER TUNING
# =============================================================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import time

# Define hyperparameter ranges for OOB validation
param_distributions = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4],
    'max_features': ['sqrt', 'log2']
}

# Create Random Forest regressor
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1, oob_score=True)

# Perform randomized search for Random Forest hyperparameter tuning
print("Performing randomized search for Random Forest hyperparameter tuning...")
print("Using out-of-bag samples for validation...")

start_time = time.time()

# Use RandomizedSearchCV for faster hyperparameter tuning
# Test 20 random combinations instead of all combinations (48 total)
random_search = RandomizedSearchCV(
    estimator=rf_regressor,
    param_distributions=param_distributions,
    n_iter=20,  # Test 20 random combinations instead of all 48
    cv=3,       # Reduced from 5 to 3 folds for faster execution
    scoring='r2',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train, y_train)

# Get best parameters and model
best_params = random_search.best_params_
best_score = random_search.best_score_
best_model = random_search.best_estimator_

# Print best parameters and score
print(f"\nBest hyperparameters: {best_params}")
print(f"Best CV R² score: {best_score:.4f}")

# Calculate standard deviation from cross-validation results (using 3 folds for speed)
cv_scores = cross_val_score(best_model, X_train, y_train, cv=3, scoring='r2')
print(f"Standard deviation: {np.std(cv_scores):.4f}")

elapsed_time = time.time() - start_time
print(f"Randomized search completed in {elapsed_time:.2f} seconds")

Performing randomized search for Random Forest hyperparameter tuning...
Using out-of-bag samples for validation...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


Python(78000) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78001) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78002) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78003) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78004) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78005) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78006) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(78007) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while


Best hyperparameters: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 10}
Best CV R² score: 0.6475
Standard deviation: 0.0205
Randomized search completed in 881.00 seconds


In [ ]:
# =============================================================================
# RANDOM FOREST MODEL - For ML Model Comparisons
# =============================================================================

# Use the best model from OOB validation
best_rf_model = best_model

# Generate predictions using the optimized Random Forest model
y_train_pred = best_rf_model.predict(X_train)
y_test_pred = best_rf_model.predict(X_test)

# Performance evaluation function for model metrics
def print_model_performance(y_true, y_pred, model_name, dataset):
    """
    Calculate and display performance metrics for regression models.
    
    Parameters:
    - y_true: actual values
    - y_pred: predicted values
    - model_name: model name for display
    - dataset: dataset name (Train/Test)
    """
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} ({dataset}): R²={r2:.4f} | RMSE={rmse:.4f} | MAE={mae:.4f}")

# Display Random Forest performance results
print("RANDOM FOREST PERFORMANCE:")
print_model_performance(y_train, y_train_pred, "Random Forest", "Train")
print_model_performance(y_test, y_test_pred, "Random Forest", "Test")

RANDOM FOREST PERFORMANCE:
Random Forest (Train): R²=0.7511 | RMSE=0.5287 | MAE=0.1763
Random Forest (Test): R²=0.6087 | RMSE=0.6100 | MAE=0.2103


In [18]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

import pandas as pd

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("FEATURE IMPORTANCE RANKING:")
print("="*50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Print model details
print(f"\nMODEL DETAILS:")
print(f"N Estimators: {best_rf_model.n_estimators}")
print(f"Max Depth: {best_rf_model.max_depth}")
print(f"Min Samples Split: {best_rf_model.min_samples_split}")
print(f"Min Samples Leaf: {best_rf_model.min_samples_leaf}")
print(f"Max Features: {best_rf_model.max_features}")
print(f"Number of Features: {best_rf_model.n_features_in_}")
print(f"Out-of-bag Score: {best_rf_model.oob_score_ if hasattr(best_rf_model, 'oob_score_') and best_rf_model.oob_score else 'Not available'}")

FEATURE IMPORTANCE RANKING:
 1. ln_at_t                   0.4475
 2. OCF_Scaled_Lag_t          0.1973
 3. NI_Scaled_t               0.1883
 4. Accruals_Scaled_t         0.0672
 5. Delta_AP_Scaled_t         0.0374
 6. Delta_Rec_Scaled_t        0.0304
 7. DP_Scaled_t               0.0263
 8. Delta_Inv_Scaled_t        0.0056

MODEL DETAILS:
N Estimators: 200
Max Depth: 10
Min Samples Split: 10
Min Samples Leaf: 4
Max Features: log2
Number of Features: 8
Out-of-bag Score: 0.6463287747087284
